In [1]:
from picamera.array import PiYUVArray
from picamera import PiCamera

from scipy.signal import find_peaks, butter, filtfilt
from pwm import PWM
import threading
import ipywidgets as ipw
import time
# import matplotlib.pyplot as plt
import skimage as ski

WIDTH_ = 224
HEIGHT_ = 160
MEDIAN_ = WIDTH_ // 2

# Camera resolution
res = (WIDTH_, HEIGHT_)

# Run a track detection algorithm on a single horizontal line.
# Uses YUV420 image format as the Y component corresponds to image intensity (gray image)
# and thus there is no need to convert from RGB to BW

camera = PiCamera()
        
# Check the link below for the combinations between mode and resolution
# https://picamera.readthedocs.io/en/release-1.13/fov.html#sensor-modes
camera.sensor_mode = 7
camera.resolution = res
camera.framerate = 120

# To filter the noise in the image we use a 3rd order Butterworth filter

# Wn = 0.02, the cut-off frequency, acceptable values are from 0 to 1
b, a = butter(3, 0.02)

pwm0 = PWM(0)
pwm1 = PWM(1)

pwm0.period = 20000000
pwm1.period = 20000000

pwm0.enable = True
pwm1.enable = True

In [2]:
pwm0.duty_cycle = 1000000
pwm1.duty_cycle = 1600000
time.sleep(1)
pwm1.duty_cycle = 1100000
time.sleep(1)
pwm1.duty_cycle = 2100000
time.sleep(1)
pwm1.duty_cycle = 1600000

In [3]:
# max speed that this algo works
# ucla = 1120000
# sonic = 1250000

UCLA_         =  1139000
USC_          =  1250000
SONIC_        =  1250000

UPPER_LINE_ = HEIGHT_ - 1
CENTER_LINE_ = HEIGHT_ // 2
LOWER_LINE_ = 0
INTERVAL_ = 0.0
CENTER_THRESHOLD_ = WIDTH_ // 12
NO_PEAKS_ = 0

STRAIGHT_ = 1600000
RIGHT_ = 1100000
LEFT_ = 2100000
NUM_PREV_VAL_ = 3

killSwitch = ipw.ToggleButton(value=False, description='Shut it down')
# minMaxSlider = ipw.IntRangeSlider(value=[UCLA_/1000,SONIC_/1000], min = 1000, max = 1500,step = 1, description = 'Speed Ranges')

# display(minMaxSlider)
display(killSwitch)

'''def within_threshold(peak):
    return abs(peak - MEDIAN_) <= CENTER_THRESHOLD_

def abs_from_straight(degree):
    return abs(degree - STRAIGHT_)'''

def main_loop():
    raw_capture = PiYUVArray(camera, size=res)
    stream = camera.capture_continuous(raw_capture, format="yuv", use_video_port=True)

    # for pwm1
    # duty_cycle

    #      (array([482, 542], dtype=int32), {'peak_heights': array([186.831036  , 185.01286929])})

    num_frames = 0

    # if line_pos is 1 then the line is to the right of the car
    # if line_pos is -1 then the line is to the left of the car
    # if line_pos is 0 either uhoh we're doomed
    # if lost is true then the line is lost
    line_pos = STRAIGHT_
    lost = False
    power5_available = False
    warp_drive = False
    prev_steer_dc = pwm1.duty_cycle
    prev_steer_diff_arr = []

    for i in range(NUM_PREV_VAL_):
        prev_steer_diff_arr.append(-1)
    
    for f in stream:
        t = time.time()

        # This was intended change speeds as necesary
        # But not anymore since its too volatile, can be manually updated
        # speeds = update_speeds()
        # UCLA_ = speeds[0]
        # SONIC_ = speeds[1]

        # If killSwitch is toggled, breaks out of forever loop and kills the motor
        if killSwitch.value:
            break

        num_frames += 1
        # Get the intensity component of the image (a trick to get black and white images)
        I = f.array[:, :, 0]
        # Reset the buffer for the next image
        raw_capture.truncate(0)
        # Select a horizontal line in the middle of the image
        u = I[UPPER_LINE_, :]
        c = I[CENTER_LINE_, :]
        l = I[LOWER_LINE_, :]
        # Smooth the transitions so we can detect the peaks 
        uf = filtfilt(b, a, u)
        cf = filtfilt(b, a, c)
        lf = filtfilt(b, a, l)
        # Find peaks which are higher than 0.5
        up = find_peaks(uf, height=128)
        cp = find_peaks(cf, height=128)
        lp = find_peaks(lf, height=128)
        
        num_up = len(up[0])
        num_cp = len(cp[0])
        num_lp = len(lp[0])

        upper_peak = NO_PEAKS_
        center_peak = NO_PEAKS_
        lower_peak = NO_PEAKS_
        
        for peak in up[0]:
            upper_peak += peak

        for peak in cp[0]:
            center_peak += peak
            
        for peak in lp[0]:
            lower_peak += peak

        if num_up != 0:
            upper_peak = upper_peak / num_up
            
        if num_cp != 0:
            center_peak = center_peak / num_cp
            # Beginning or End
            # if num_cp == 3:
            #   print("THIS IS THE BEGINNING OF THE END")
            
        if num_lp != 0:
            lower_peak = lower_peak / num_lp

        if center_peak != NO_PEAKS_:
            if center_peak < MEDIAN_:
                if lost and line_pos == RIGHT_:
                    # If this happens, that means that the center line is to the right
                    # But we encountered the bordering track to the left
                    # Break to stop going over and fucking shit up
                    # TODO: Make clause that handles this, so in case of race it doesn't stop without violating shit
                    break
                    # pwm1.duty_cycle = RIGHT_
                    # time.sleep(0.1)
                    # continue
                line_pos = LEFT_
            elif center_peak > MEDIAN_:
                if lost and line_pos == LEFT_:
                    # If this happens, that means that the center line is to the left
                    # But we encountered the bordering track to the right
                    # Break to stop going over and fucking shit up
                    # TODO: Make clause that handles this, so in case of race it doesn't stop without violating shit
                    break
                    # pwm1.duty_cycle = LEFT_
                    # time.sleep(0.1)
                    # continue;
                line_pos = RIGHT_
            else:
                # chances are this isn't going to happen but whatever
                # can stop the turning if the track is too
                line_pos = STRAIGHT_

            # Sets the steering position based on the peak of the Center line
            if not warp_drive:
                pwm1.duty_cycle = int(STRAIGHT_ - ((center_peak - MEDIAN_)/MEDIAN_)*500000)
            else:
                pwm1.duty_cycle = int(STRAIGHT_ - ((center_peak - MEDIAN_)/MEDIAN_)*200000)
            lost = False
        else:
            if line_pos == LEFT_:
                pwm1.duty_cycle = LEFT_
            elif line_pos == RIGHT_:
                pwm1.duty_cycle = RIGHT_
            else:
                pwm1.duty_cycle = STRAIGHT_

            lost = True
            # pwm0.duty_cycle = SONIC_
            
        # for overcorrecting
        
        '''if upper_peak != NO_PEAKS_ and lower_peak != NO_PEAKS_:
            # turning left but upper shows more to right
            if pwm1.duty_cycle > STRAIGHT_ and upper_peak > lower_peak:
                print("OVER CORRECTING LEFT, TURNING RIGHT")
                pwm1.duty_cycle -= int((pwm1.duty_cycle - STRAIGHT_) * 0.05)
            elif pwm1.duty_cycle < STRAIGHT_ and upper_peak < lower_peak:
                print("OVER CORRECTING RIGHT, TURNING LEFT")
                pwm1.duty_cycle += int((STRAIGHT_ - pwm1.duty_cycle) * 0.05)'''
            
        ###################
        # DONE WITH TURNS #
        ###################
        
        i = NUM_PREV_VAL_ - 1
        while i > 0:
            prev_steer_diff_arr[i] = prev_steer_diff_arr[i - 1]
            i -= 1
        
        prev_steer_diff_arr[0] = abs(pwm1.duty_cycle - STRAIGHT_) - abs(prev_steer_dc - STRAIGHT_)
        prev_steer_dc = pwm1.duty_cycle

        if all(i > 0 for i in prev_steer_diff_arr) and power5_available and pwm1.duty_cycle > 400000:
            pwm0.duty_cycle = SONIC_
            '''if pwm1.duty_cycle > STRAIGHT_:
                pwm1.duty_cycle = LEFT_
            if pwm1.duty_cycle < STRAIGHT_:
                pwm1.duty_cycle = RIGHT_'''
            print("SONIC")
            power5_available = False
        elif upper_peak != NO_PEAKS_ and center_peak != NO_PEAKS_ and lower_peak != NO_PEAKS_:
            if abs(upper_peak - MEDIAN_) <= CENTER_THRESHOLD_ and abs(center_peak - MEDIAN_) <= CENTER_THRESHOLD_ and abs(lower_peak - MEDIAN_) <= CENTER_THRESHOLD_:
                pwm0.duty_cycle = USC_
                print("WARP")
                warp_drive = True
        else:
            '''if warp_drive and upper_peak != NO_PEAKS_:
                if upper_peak > MEDIAN_:
                    pwm1.duty_cycle = RIGHT_ - 250000
                else:
                    pwm1.duty_cycle = LEFT_ - 250000
            warp_drive = False'''    
            pwm0.duty_cycle = UCLA_
            """if abs(pwm1.duty_cycle - STRAIGHT_) > 300000:
                pwm0.duty_cycle = UCLA_ + 4000"""
        
        if abs(center_peak - MEDIAN_) <= CENTER_THRESHOLD_:
            power5_available = True

        time_elapsed = time.time() - t
        
        print("Elapsed {:0.4f} seconds, estimated FPS {:0.2f}".format(time_elapsed, 1 / time_elapsed))
        time.sleep(INTERVAL_)
        
    pwm0.duty_cycle = 1000000

    # Release resources
    stream.close()
    raw_capture.close()
    camera.close()

ToggleButton(value=False, description='Shut it down')

Elapsed 0.0096 seconds, estimated FPS 104.20
Elapsed 0.0100 seconds, estimated FPS 100.16


In [4]:
t = threading.Thread(target=main_loop)
t.start()

Elapsed 0.0107 seconds, estimated FPS 93.89
Elapsed 0.0094 seconds, estimated FPS 106.40
Elapsed 0.0094 seconds, estimated FPS 106.51
Elapsed 0.0094 seconds, estimated FPS 106.17
Elapsed 0.0094 seconds, estimated FPS 106.69
Elapsed 0.0094 seconds, estimated FPS 106.74
Elapsed 0.0095 seconds, estimated FPS 105.68
Elapsed 0.0094 seconds, estimated FPS 106.51
Elapsed 0.0094 seconds, estimated FPS 106.33
Elapsed 0.0094 seconds, estimated FPS 106.48
Elapsed 0.0092 seconds, estimated FPS 108.53
Elapsed 0.0093 seconds, estimated FPS 107.85
Elapsed 0.0093 seconds, estimated FPS 107.63
Elapsed 0.0195 seconds, estimated FPS 51.25
Elapsed 0.0092 seconds, estimated FPS 108.44
Elapsed 0.0092 seconds, estimated FPS 108.61
Elapsed 0.0092 seconds, estimated FPS 109.05
Elapsed 0.0090 seconds, estimated FPS 111.05
Elapsed 0.0092 seconds, estimated FPS 108.37
Elapsed 0.0092 seconds, estimated FPS 109.16
Elapsed 0.0093 seconds, estimated FPS 106.99
Elapsed 0.0094 seconds, estimated FPS 106.55
Elapsed 0.00

Elapsed 0.0096 seconds, estimated FPS 104.60
Elapsed 0.0091 seconds, estimated FPS 109.80
Elapsed 0.0091 seconds, estimated FPS 110.20
Elapsed 0.0092 seconds, estimated FPS 108.31
Elapsed 0.0091 seconds, estimated FPS 109.35
Elapsed 0.0091 seconds, estimated FPS 109.83
Elapsed 0.0091 seconds, estimated FPS 109.35
Elapsed 0.0092 seconds, estimated FPS 109.05
Elapsed 0.0091 seconds, estimated FPS 110.05
Elapsed 0.0092 seconds, estimated FPS 108.90
Elapsed 0.0091 seconds, estimated FPS 109.39
Elapsed 0.0092 seconds, estimated FPS 108.46
Elapsed 0.0091 seconds, estimated FPS 109.57
Elapsed 0.0092 seconds, estimated FPS 109.03
Elapsed 0.0092 seconds, estimated FPS 108.74
Elapsed 0.0092 seconds, estimated FPS 108.47
Elapsed 0.0092 seconds, estimated FPS 108.51
Elapsed 0.0092 seconds, estimated FPS 109.22
Elapsed 0.0092 seconds, estimated FPS 108.35
Elapsed 0.0092 seconds, estimated FPS 108.71
Elapsed 0.0091 seconds, estimated FPS 109.35
Elapsed 0.0091 seconds, estimated FPS 110.33
Elapsed 0.

Elapsed 0.0092 seconds, estimated FPS 108.98
Elapsed 0.0094 seconds, estimated FPS 106.37
Elapsed 0.0091 seconds, estimated FPS 110.12
Elapsed 0.0092 seconds, estimated FPS 108.78
Elapsed 0.0092 seconds, estimated FPS 109.21
Elapsed 0.0092 seconds, estimated FPS 109.02
Elapsed 0.0091 seconds, estimated FPS 109.49
Elapsed 0.0092 seconds, estimated FPS 108.72
Elapsed 0.0092 seconds, estimated FPS 108.89
Elapsed 0.0092 seconds, estimated FPS 108.22
Elapsed 0.0094 seconds, estimated FPS 106.47
Elapsed 0.0093 seconds, estimated FPS 107.12
Elapsed 0.0093 seconds, estimated FPS 107.48
Elapsed 0.0094 seconds, estimated FPS 106.82
Elapsed 0.0094 seconds, estimated FPS 106.85
Elapsed 0.0094 seconds, estimated FPS 106.65
Elapsed 0.0093 seconds, estimated FPS 107.26
Elapsed 0.0093 seconds, estimated FPS 107.08
Elapsed 0.0094 seconds, estimated FPS 106.68
Elapsed 0.0093 seconds, estimated FPS 107.27
Elapsed 0.0137 seconds, estimated FPS 72.81
Elapsed 0.0094 seconds, estimated FPS 106.27
Elapsed 0.0

Elapsed 0.0098 seconds, estimated FPS 101.82
Elapsed 0.0093 seconds, estimated FPS 107.89
Elapsed 0.0092 seconds, estimated FPS 108.83
Elapsed 0.0092 seconds, estimated FPS 108.93
Elapsed 0.0092 seconds, estimated FPS 108.70
Elapsed 0.0092 seconds, estimated FPS 108.95
Elapsed 0.0132 seconds, estimated FPS 76.00
Elapsed 0.0092 seconds, estimated FPS 108.35
Elapsed 0.0090 seconds, estimated FPS 111.19
Elapsed 0.0093 seconds, estimated FPS 107.99
Elapsed 0.0092 seconds, estimated FPS 108.89
Elapsed 0.0092 seconds, estimated FPS 109.07
Elapsed 0.0100 seconds, estimated FPS 99.79
Elapsed 0.0093 seconds, estimated FPS 107.69
Elapsed 0.0092 seconds, estimated FPS 109.27
Elapsed 0.0092 seconds, estimated FPS 108.57
Elapsed 0.0093 seconds, estimated FPS 108.03
Elapsed 0.0091 seconds, estimated FPS 109.69
Elapsed 0.0132 seconds, estimated FPS 75.74
Elapsed 0.0092 seconds, estimated FPS 108.71
Elapsed 0.0092 seconds, estimated FPS 108.41
Elapsed 0.0092 seconds, estimated FPS 109.16
Elapsed 0.009

Elapsed 0.0109 seconds, estimated FPS 91.67
Elapsed 0.0093 seconds, estimated FPS 107.21
Elapsed 0.0092 seconds, estimated FPS 108.24
Elapsed 0.0095 seconds, estimated FPS 105.65
Elapsed 0.0095 seconds, estimated FPS 105.09
Elapsed 0.0094 seconds, estimated FPS 106.68
Elapsed 0.0100 seconds, estimated FPS 100.37
Elapsed 0.0093 seconds, estimated FPS 107.81
Elapsed 0.0094 seconds, estimated FPS 106.95
Elapsed 0.0093 seconds, estimated FPS 107.21
Elapsed 0.0094 seconds, estimated FPS 106.38
Elapsed 0.0091 seconds, estimated FPS 109.46
Elapsed 0.0132 seconds, estimated FPS 75.56
Elapsed 0.0093 seconds, estimated FPS 107.43
Elapsed 0.0091 seconds, estimated FPS 109.99
Elapsed 0.0091 seconds, estimated FPS 109.32
Elapsed 0.0093 seconds, estimated FPS 107.25
Elapsed 0.0094 seconds, estimated FPS 106.60
Elapsed 0.0100 seconds, estimated FPS 99.72
Elapsed 0.0092 seconds, estimated FPS 108.35
Elapsed 0.0093 seconds, estimated FPS 107.25
Elapsed 0.0093 seconds, estimated FPS 107.89
Elapsed 0.009

Elapsed 0.0097 seconds, estimated FPS 103.46
Elapsed 0.0095 seconds, estimated FPS 105.33
Elapsed 0.0095 seconds, estimated FPS 104.97
Elapsed 0.0098 seconds, estimated FPS 101.81
Elapsed 0.0096 seconds, estimated FPS 104.33
Elapsed 0.0095 seconds, estimated FPS 105.37
Elapsed 0.0106 seconds, estimated FPS 94.36
Elapsed 0.0095 seconds, estimated FPS 105.14
Elapsed 0.0094 seconds, estimated FPS 106.25
Elapsed 0.0094 seconds, estimated FPS 105.97
Elapsed 0.0096 seconds, estimated FPS 104.42
Elapsed 0.0095 seconds, estimated FPS 105.39
Elapsed 0.0105 seconds, estimated FPS 95.27
Elapsed 0.0095 seconds, estimated FPS 104.79
Elapsed 0.0095 seconds, estimated FPS 104.84
Elapsed 0.0093 seconds, estimated FPS 107.74
Elapsed 0.0094 seconds, estimated FPS 106.31
Elapsed 0.0153 seconds, estimated FPS 65.51
Elapsed 0.0092 seconds, estimated FPS 108.14
Elapsed 0.0091 seconds, estimated FPS 110.16
Elapsed 0.0093 seconds, estimated FPS 107.28
Elapsed 0.0093 seconds, estimated FPS 107.02
Elapsed 0.009

In [1]:
from pwm import PWM

pwm0 = PWM(0)

pwm0.duty_cycle = 1000000

pwm0.period = 20000000

pwm0.enable = True